In [1]:
import pandas as pd
import random

In [2]:
LIST_FILE_PATH = "./lstm_preds/results.csv" #"#"./lstm_preds/results_K0.csv"
LIST_FILE_PATH_2 = "./lstm_preds/results_on_test.csv"

In [3]:
%%time
df = pd.read_csv(LIST_FILE_PATH,usecols=[0,1,2,4])
df.rename(columns={"pred":"city_id"},inplace=True)
df["rank"] = df.groupby("utrip_id").transform("cumcount")
###
print(df.loc[df.label==1]["rank"] .describe(percentiles=[.25,.5,.75,.9,.95,.99]))
df["score_norm"] = df["score"].div(df.groupby("rank")["score"].transform("mean")).round(3)
df =  df.loc[df["city_id"]>-1] ## note, different eval/ranks
df.loc[df["score"]==0,["score","rank"]] = np.nan
df = df.fillna(method="ffill") ## impute values from last valid for hand inserted 1s (that lack a score)
####

display(df)

count    195341.000000
mean         12.560830
std          16.861288
min           0.000000
25%           1.000000
50%           4.000000
75%          17.000000
90%          50.000000
95%          50.000000
99%          50.000000
max          50.000000
Name: rank, dtype: float64


,utrip_id,city_id,score,label,rank,score_norm
0,1000027_1,65365,4.966359,0,0.0,0.611
1,1000027_1,60902,4.748976,0,1.0,0.622
2,1000027_1,58178,4.671995,0,2.0,0.635
3,1000027_1,58015,4.573554,0,3.0,0.640
4,1000027_1,5385,4.562560,0,4.0,0.653
...,...,...,...,...,...,...
9789767,999755_1,47378,3.726496,0,45.0,0.734
9789768,999755_1,66414,3.662008,0,46.0,0.724
9789769,999755_1,62185,3.653897,0,47.0,0.725
9789770,999755_1,56198,3.651016,0,48.0,0.727


Wall time: 9.05 s


In [4]:
df["label"].sum()

191032

* Impute values for labels missing a prediction score, rank, etc' 

In [5]:
df.loc[df.label==1].tail(9)

,utrip_id,city_id,score,label,rank,score_norm
9789321,998444_2,2416,8.337121,1,0.0,1.026
9789371,998756_1,66648,8.183451,1,0.0,1.007
9789471,998814_1,63263,2.860102,1,49.0,0.572
9789472,999122_1,6582,9.742151,1,0.0,1.199
9789523,999341_2,9699,4.109669,1,1.0,0.539
9789577,999385_2,8344,7.451572,1,5.0,1.088
9789626,999583_3,47700,6.152070,1,4.0,0.881
9789673,999674_3,41168,5.678018,1,1.0,0.744
9789726,999755_1,9608,5.296478,1,4.0,0.758


In [6]:
# df.loc[(df.label==1) & (df["rank"]>40)][["score","label","rank","score_norm"]].describe()
# mean score: 5.1 , mean rank : 38, mean score_norm 0.978
## over 40,  mean score: 4.95 , mean rank : 44, mean score_norm 0.978
###
# df.loc[(df.label==0) & (df["rank"]>30)][["score","label","rank","score_norm"]].describe()
### mean score: 5.2 , median: 5.29
### median rank: 40
### mean score_norm: 1.06 , median : 1.02

In [7]:
df.loc[df["score"].isna()]

,utrip_id,city_id,score,label,rank,score_norm


In [8]:
df.loc[df.label==1]["rank"] .describe(percentiles=[.25,.5,.75,.9,.95,.99])

count    191032.000000
mean         11.619948
std          15.845896
min           0.000000
25%           1.000000
50%           4.000000
75%          15.000000
90%          48.000000
95%          49.000000
99%          49.000000
max          49.000000
Name: rank, dtype: float64

In [9]:
df.loc[(df.label==1) & (df["rank"]<50)]["rank"] .describe(percentiles=[.25,.5,.75,.9,.95,.99])

count    191032.000000
mean         11.619948
std          15.845896
min           0.000000
25%           1.000000
50%           4.000000
75%          15.000000
90%          48.000000
95%          49.000000
99%          49.000000
max          49.000000
Name: rank, dtype: float64

In [10]:
%%time
## test/candidates
df2 = pd.read_csv("./lstm_preds/results_on_test.csv")
df2.rename(columns={"pred":"city_id"},inplace=True)
df2["rank"] = df2.groupby("utrip_id").transform("cumcount")
# ###
print(df2.shape[0])
df2 =  df2.loc[df2["city_id"]>-1] ## note, different eval/ranks
print(df2.shape[0])
df2.loc[df2["score"]==0,["score","rank"]] = np.nan
####
df2["score_norm"] = df2["score"].div(df2.groupby("rank")["score"].transform("mean")).round(3)
print("max rank",df2["rank"].max())
display(df2)

7066200
7066200
max rank 99.0


,utrip_id,city_id,score,rank,score_norm
0,1000066_2,5797,9.486283,0.0,1.145
1,1000066_2,29058,8.458694,1.0,1.087
2,1000066_2,41971,8.300508,2.0,1.107
3,1000066_2,39200,8.270500,3.0,1.134
4,1000066_2,46854,8.229909,4.0,1.154
...,...,...,...,...,...
7066195,999991_3,34895,3.939668,95.0,0.879
7066196,999991_3,42368,3.937382,96.0,0.881
7066197,999991_3,46680,3.936581,97.0,0.882
7066198,999991_3,20866,3.933790,98.0,0.884


Wall time: 4.14 s


### downsample


In [11]:
%%time
df_list_1 = df.loc[(df["rank"]<=15) | (df["label"]==1)]    
df_list_2 = df.loc[(df["rank"]>15) & (df["label"]==0)].groupby(["utrip_id"]).sample(n=11,random_state=42)
### TEMP! 
df2 = df2.loc[df2["rank"]<54]
###
df = pd.concat([df_list_1,df_list_2,df2])
print(df.dtypes)
del df_list_1,df_list_2

utrip_id       object
city_id         int64
score         float64
label         float64
rank          float64
score_norm    float64
dtype: object
Wall time: 2min 5s


In [12]:
%%time
df = df.apply(pd.to_numeric, errors='ignore',downcast="float").apply(pd.to_numeric, errors='ignore',downcast="integer")
print(df.dtypes)

utrip_id       object
city_id         int32
score         float32
label         float32
rank             int8
score_norm    float32
dtype: object
Wall time: 2.17 s


In [13]:
df.city_id.describe()

count    9.137676e+06
mean     3.314400e+04
std      1.955428e+04
min      5.000000e+00
25%      1.604700e+04
50%      3.258600e+04
75%      5.026900e+04
max      6.756500e+04
Name: city_id, dtype: float64

In [14]:
df.loc[df.city_id.isna()]

,utrip_id,city_id,score,label,rank,score_norm


In [15]:
# %%time
# n_users = df["utrip_id"].nunique()
# print("n_users",n_users)

# res = 100*df.sort_values(["utrip_id","rank"],ascending=True).groupby("utrip_id")["label"].head(4).sum() / n_users
# print("top4 acc: by model {0:.4f}".format(res))

In [16]:
df

,utrip_id,city_id,score,label,rank,score_norm
0,1000027_1,65365,4.966359,0.0,0,0.611
1,1000027_1,60902,4.748976,0.0,1,0.622
2,1000027_1,58178,4.671995,0.0,2,0.635
3,1000027_1,58015,4.573554,0.0,3,0.640
4,1000027_1,5385,4.562560,0.0,4,0.653
...,...,...,...,...,...,...
7066149,999991_3,67088,4.296225,NaN,49,0.841
7066150,999991_3,67223,4.262345,NaN,50,0.837
7066151,999991_3,33195,4.259714,NaN,51,0.840
7066152,999991_3,54474,4.258322,NaN,52,0.843


In [17]:
df.to_parquet("all_candidates_list.parquet",index=False)